In [ ]:
"E:\RAGexpHMS\data\text_files"

In [23]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
loader = DirectoryLoader(
    r"E:\RAGexpHMS\data\text_files",
    glob="**/*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"}
)

documents = loader.load()
documents

[Document(metadata={'source': 'E:\\RAGexpHMS\\data\\text_files\\ac_failure_room.txt'}, page_content='\n\n**Incident Title:** Air Conditioning Failure – Room 204\n**Location:** Hotel Sunrise Coastal Resort\n**Duration of Issue:** 3 Days\n**Status:** Guest departed unsatisfied\n**Category:** Maintenance failure / Guest comfort\n\n---\n\n### **Overview**\n\nThis document contains a detailed fictional complaint account about a prolonged air-conditioning failure in Room 204 and the resulting consequences. The narrative describes guest interaction, technical issues, staff responses, and emotional impact. It is intentionally lengthy for RAG/document retrieval testing.\n\n---\n\n### **Day 1 – Arrival Experience**\n\nI checked into Room 204 on a hot summer afternoon at around 3:30 PM. The lobby temperature felt perfectly cool, so I assumed the guest rooms would be the same. However, when I opened the door to Room 204, a wave of warm, stuffy air hit my face. The room temperature felt well above 

In [24]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

pdf_loader = DirectoryLoader(
    r"E:\RAGexpHMS\data\pdf",
    glob="**/*.pdf",
    loader_cls=PyPDFLoader,
)

pdf_docs = pdf_loader.load()
pdf_docs


[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-12-20T19:35:42+05:30', 'author': 'Aman Soni', 'moddate': '2025-12-20T19:35:42+05:30', 'source': 'E:\\RAGexpHMS\\data\\pdf\\cab_charges.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='This document outlines the cab and transportation charges followed in hotel \nmanagement operations. Hotels often provide transportation services, including airport \ntransfers, local sightseeing, intercity travel, and chauffeur-driven cabs. Clear cab \ncharge policies ensure transparency, proper billing, and guest satisfaction while \nmaintaining operational efficiency. \n1. Types of Cab Services: \nHotels typically offer various types of cab services: \n• Airport Transfers: Pick-up and drop services to and from airports. Charges may \nvary based on distance, vehicle type, and timing. \n• Local Transportation: Cabs for city tours, shopping, or loca

In [44]:
from langchain_community.document_loaders import DirectoryLoader, CSVLoader

csv_loader = DirectoryLoader(
    r"E:\RAGexpHMS\data\csv",
    glob="**/*.csv",
    loader_cls=CSVLoader,
)

csv_docs = csv_loader.load()
csv_docs

[Document(metadata={'source': 'E:\\RAGexpHMS\\data\\csv\\bookings.csv', 'row': 0}, page_content='booking_id: B001\nguest_id: G001\nguest_name: John Smith\nroom_number: 101\nroom_type: Deluxe\ncheck_in_date: 2025-01-05\ncheck_out_date: 2025-01-07\nbooking_status: Confirmed\nbooking_source: Website\ntotal_amount: 4500\npayment_status: Paid\ncancellation_reason: '),
 Document(metadata={'source': 'E:\\RAGexpHMS\\data\\csv\\bookings.csv', 'row': 1}, page_content='booking_id: B002\nguest_id: G002\nguest_name: Emma Brown\nroom_number: 102\nroom_type: Standard\ncheck_in_date: 2025-01-06\ncheck_out_date: 2025-01-08\nbooking_status: Cancelled\nbooking_source: MobileApp\ntotal_amount: 3200\npayment_status: Refunded\ncancellation_reason: Change of plans'),
 Document(metadata={'source': 'E:\\RAGexpHMS\\data\\csv\\bookings.csv', 'row': 2}, page_content='booking_id: B003\nguest_id: G003\nguest_name: Michael Johnson\nroom_number: 103\nroom_type: Suite\ncheck_in_date: 2025-01-07\ncheck_out_date: 2025-0

In [45]:
# Combine all loaded docs into a single list. in future when we add more loader then we will adding them to here.

combined_docs = []


try:
    combined_docs += documents
except:
    pass


try:
    combined_docs += pdf_docs
except:
    pass


try:
    combined_docs += csv_docs
except:
    pass


try:
    combined_docs += docx_docs
except:
    pass

print("Documents combined:", len(combined_docs))


Documents combined: 293


In [46]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)

chunks = text_splitter.split_documents(combined_docs)

print(f"Chunks created: {len(chunks)}")
print(chunks[0].page_content[:300])
chunks

Chunks created: 661
**Incident Title:** Air Conditioning Failure – Room 204
**Location:** Hotel Sunrise Coastal Resort
**Duration of Issue:** 3 Days
**Status:** Guest departed unsatisfied
**Category:** Maintenance failure / Guest comfort

---

### **Overview**


[Document(metadata={'source': 'E:\\RAGexpHMS\\data\\text_files\\ac_failure_room.txt'}, page_content='**Incident Title:** Air Conditioning Failure – Room 204\n**Location:** Hotel Sunrise Coastal Resort\n**Duration of Issue:** 3 Days\n**Status:** Guest departed unsatisfied\n**Category:** Maintenance failure / Guest comfort\n\n---\n\n### **Overview**'),
 Document(metadata={'source': 'E:\\RAGexpHMS\\data\\text_files\\ac_failure_room.txt'}, page_content='---\n\n### **Overview**\n\nThis document contains a detailed fictional complaint account about a prolonged air-conditioning failure in Room 204 and the resulting consequences. The narrative describes guest interaction, technical issues, staff responses, and emotional impact. It is intentionally lengthy for RAG/document retrieval testing.\n\n---\n\n### **Day 1 – Arrival Experience**'),
 Document(metadata={'source': 'E:\\RAGexpHMS\\data\\text_files\\ac_failure_room.txt'}, page_content='---\n\n### **Day 1 – Arrival Experience**\n\nI checked in

In [47]:
import numpy as np
from sentence_transformers import SentenceTransformer
# import transformers
# # import sentence_transformers
import chromadb
from chromadb.config import Settings
import uuid
from typing import List,Dict,Any,Tuple
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [48]:
class EmbeddingManager:
    """Handles document embeedding generation using senetenceTransformer"""

    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Intialize the embedding manager

        Args:
            model_name: HuggingFace model name for sentence Embedding
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embdding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded succssfully.Embedding dimnesion: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self,texts: List[str]) -> np.ndarray:
        """  
        Generate embeddings for a list of texts

        args:
        texts: list of text strings to embed

        Returns:
        numpy array of embdding with shape(len(teexts),embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        print(f"Generating embedding for {len(texts)} texts...")
        embeddings = self.model.encode(texts,show_progress_bar= True)
        print(f"Genrated embeddings with shape: {embeddings.shape}")
        return embeddings
    

#initializing the embedding managers:

embedding_manager = EmbeddingManager()
embedding_manager

Loading embdding model: all-MiniLM-L6-v2
Model loaded succssfully.Embedding dimnesion: 384


In [49]:
print(type(chunks))
print(type(chunks[0]))


<class 'list'>
<class 'langchain_core.documents.base.Document'>


In [50]:
print("docs:", len(combined_docs))
print("chunks:", len(chunks))
print(type(combined_docs))
print(type(combined_docs[0]) if len(combined_docs) > 0 else "NO DOCS")


docs: 293
chunks: 661
<class 'list'>
<class 'langchain_core.documents.base.Document'>


In [51]:
import os

In [52]:
class VectorStore:
    """Manages document embeddings in a chromDB vector store"""

    def __init__(self,collection_name: str = "pdf_docs", persist_directory: str = "../data/vector_store"):
        """ 
        Initialize the vectore space

        Args: collection_name: nam of the chormadb collection
        persist_directory: Directory to persist thee vectore store
        """

        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize Chromadb client and collection. Creates the persist directory and
        ensures a collection with `self.collection_name` exists. """

        try:
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            # print(f"Starting Chromadb client with persist dir: {self.persist_directory}")
            # settings = Settings(chroma_db_impl="duckdb+parquet", persist_directory=self.persist_directory)
            # self.client = chromadb.Client(settings=settings)
            # try to get existing collection, otherwise create it
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name, metadata={"description": "PDF documents embeddings for RAG"})
            print(f"Vector store initialized with collection: {self.collection_name}")
            print(f"existing documents in collection: {self.collection.count()}")
        except Exception as e:
            print(f"Failed to initialize vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store

        Args:
            documents: List of Document objects to add
            embeddings: Corresponding numpy array of embeddings
        """
        if len(documents) != embeddings.shape[0]:
            raise ValueError("Number of documents and embeddings must match")
        print(f"Adding {len(documents)} documents to the vector store...")

       # prepare data for chromadb
        ids = []
        metadatas = []
        documents_texts = []
        embeddings_list = []
        # for i, doc in enumerate(zip(documents,embeddings)):
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            # prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            #document content
            documents_texts.append(doc.page_content)
            embeddings_list.append(embeddings[i].tolist())
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_texts,
                
            )
            print(f"Successfully added {len(documents)} documents to the vector store.")
            print(f"Total documents in collection after addition: {self.collection.count()}")
        except Exception as e:
            print(f"Failed to add documents to vector store: {e}")
            raise

vectorstore = VectorStore()
vectorstore
    

Vector store initialized with collection: pdf_docs
existing documents in collection: 498


In [53]:
print(vectorstore)


In [54]:
#convert the text to embeddings
texts = [docs.page_content for docs in chunks]
texts

['**Incident Title:** Air Conditioning Failure – Room 204\n**Location:** Hotel Sunrise Coastal Resort\n**Duration of Issue:** 3 Days\n**Status:** Guest departed unsatisfied\n**Category:** Maintenance failure / Guest comfort\n\n---\n\n### **Overview**',
 '---\n\n### **Overview**\n\nThis document contains a detailed fictional complaint account about a prolonged air-conditioning failure in Room 204 and the resulting consequences. The narrative describes guest interaction, technical issues, staff responses, and emotional impact. It is intentionally lengthy for RAG/document retrieval testing.\n\n---\n\n### **Day 1 – Arrival Experience**',
 '---\n\n### **Day 1 – Arrival Experience**\n\nI checked into Room 204 on a hot summer afternoon at around 3:30 PM. The lobby temperature felt perfectly cool, so I assumed the guest rooms would be the same. However, when I opened the door to Room 204, a wave of warm, stuffy air hit my face. The room temperature felt well above 30°C (86°F). The air conditio

In [55]:
texts = [doc.page_content for doc in chunks]

# generate embeddings
embeddings = embedding_manager.generate_embeddings(texts)


# store in vector database
vectorstore.add_documents(chunks, embeddings)

Generating embedding for 661 texts...


Batches: 100%|██████████| 21/21 [00:10<00:00,  2.08it/s]


Genrated embeddings with shape: (661, 384)
Adding 661 documents to the vector store...
Successfully added 661 documents to the vector store.
Total documents in collection after addition: 1159


In [56]:
class RADRetriever:
    """Retrieves relevant documents from the vector store based on query embeddings"""

    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever

        Args:
            vector_store: Instance of VectorStore to retrieve documents from
            embedding_manager: Instance of EmbeddingManager to generate query embeddings
            
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager
    
    def retrieve(self, query: str, top_k: int = 5) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a given query

        Args:
            query: The input query string
            top_k: Number of top relevant documents to retrieve"""
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            #process and return results
            retrieved_docs = []
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    similarity_score = 1 - distance  # assuming distance is in [0,2] for cosine distance

                    if similarity_score >= 0:  # filter out non-relevant documents
                        retrieved_docs.append({
                            "id": doc_id,
                            "content": document,
                            "metadata": metadata,
                            "similarity_score": similarity_score,
                            "distance": distance,
                            "rank": i + 1
                    })
                        
                print(f"Retrieved {len(retrieved_docs)} relevant documents for the query.")
            else:
                print("No documents retrieved for the query.")
            return retrieved_docs
        except Exception as e:
            print(f"Failed to retrieve documents: {e}")
            return []
        
rag_retriever = RADRetriever(vectorstore, embedding_manager)
rag_retriever
        

        

In [57]:
rag_retriever.retrieve("cab service cost?", top_k=3)

Generating embedding for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  6.17it/s]

Genrated embeddings with shape: (1, 384)


Retrieved 3 relevant documents for the query.


[{'id': 'doc_4ab24ee1_69',
  'content': '12. Integration with Other Services: \nCab services often complement hotel services such as: \n• Airport meet-and-greet services. \n• Guided city tours and excursions. \n• Hotel packages including meals, sightseeing, and transportation. \nCharges for integrated packages are predefined and communicated in advance.',
  'metadata': {'page': 2,
   'creator': 'Microsoft® Word for Microsoft 365',
   'moddate': '2025-12-20T19:35:42+05:30',
   'doc_index': 69,
   'author': 'Aman Soni',
   'producer': 'Microsoft® Word for Microsoft 365',
   'content_length': 309,
   'creationdate': '2025-12-20T19:35:42+05:30',
   'total_pages': 4,
   'source': 'E:\\RAGexpHMS\\data\\pdf\\cab_charges.pdf',
   'page_label': '3'},
  'similarity_score': 0.18405592441558838,
  'distance': 0.8159440755844116,
  'rank': 1},
 {'id': 'doc_f5e429f1_320',
  'content': '12. Integration with Other Services: \nCab services often complement hotel services such as: \n• Airport meet-and-g

In [ ]:
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

In [58]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()
###intializing the groq llm:
groq_key = "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"
llm = ChatGroq(groq_api_key=groq_key, model="llama-3.1-8b-instant", temperature=0.1,max_tokens=1024)

## simple rag function rtrive + generate answer
def rag_answer(query, retriever, llm, top_k=3):
    #retrieve relevant documents
    retrieved_docs = retriever.retrieve(query, top_k=top_k)
    context = "\n\n".join([doc['content'] for doc in retrieved_docs]) if retrieved_docs else ""
    if not retrieved_docs:
        return "No relevant documents found to answer the query."

    #create the prompt for llm for genrating answer
    prompt = f"Using the following context, answer the question Concisely:\n\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"

    #generate answer using llm
    response = llm.invoke([prompt.format(context=context, query=query)])
    return response.content

In [59]:
answer = rag_answer("breakfast menu", rag_retriever, llm, top_k=3)
print("Answer:", answer)

Generating embedding for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 79.88it/s]

Genrated embeddings with shape: (1, 384)
Retrieved 3 relevant documents for the query.


Answer: Breakfast menu offerings.


In [60]:
answer = rag_answer("If I cancel 24 hours before arrival, will I be charged?", rag_retriever, llm, top_k=3)
print("Answer:", answer)



Generating embedding for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 66.67it/s]

Genrated embeddings with shape: (1, 384)


Retrieved 3 relevant documents for the query.
Answer: No, you will likely be eligible for a full or partial refund.


In [40]:
answer = rag_answer("What types of breakfast options are available at the hotel?", rag_retriever, llm, top_k=3)
print("Answer:", answer)

Generating embedding for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 64.40it/s]

Genrated embeddings with shape: (1, 384)
Retrieved 3 relevant documents for the query.


Answer: The hotel offers various types of breakfast services, including buffet, à la carte, continental, and in-room dining options.


In [42]:
answer = rag_answer("What is food menu available in lunch?", rag_retriever, llm, top_k=3)
print("Answer:", answer)

Generating embedding for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 55.59it/s]

Genrated embeddings with shape: (1, 384)
Retrieved 3 relevant documents for the query.


Answer: The context does not specifically mention the types of food available in a lunch menu. However, based on general knowledge, a typical lunch menu may include items such as sandwiches, salads, soups, pasta, burgers, and various international dishes.


In [43]:
answer = rag_answer("How are refunds processed if I book via a travel agency?", rag_retriever, llm, top_k=3)
print("Answer:", answer)

Generating embedding for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 52.63it/s]

Genrated embeddings with shape: (1, 384)


Retrieved 3 relevant documents for the query.
Answer: If you book via a travel agency, refunds are usually processed through the agency itself, not directly with the hotel.


In [62]:
answer = rag_answer("How many rooms are currently available?", rag_retriever, llm, top_k=3)
print("Answer:", answer)

Generating embedding for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 12.12it/s]

Genrated embeddings with shape: (1, 384)
Retrieved 3 relevant documents for the query.


Answer: There is 1 room available.


In [67]:
# How many guests have Platinum loyalty status?
answer = rag_answer("List all incidents related to staff misconduct.", rag_retriever, llm, top_k=3)
print("Answer:", answer)

Generating embedding for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 90.96it/s]

Genrated embeddings with shape: (1, 384)


Retrieved 3 relevant documents for the query.
Answer: 1. I005: Staff behavior was rude at reception.
